# Decompose MOB data

Apply NMF or sICA factorization to MOB data

In [3]:
import sys
import os
curdir = os.path.abspath(os.path.curdir)
sys.path.append(os.path.join(curdir,'FUImaging'))

In [4]:
from configobj import ConfigObj
import glob
import numpy as np
from regnmf import ImageAnalysisComponents as ia
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
from PIL import Image

### Specify parameter

In [5]:
basepath = os.path.join('/media/jan/BackupWork/Documents/NewAnalysis')
datapath = os.path.join(basepath, 'MOBconverted') #where to get the data
savepath = os.path.join(basepath, 'MOBdecomposed') #where to save decompostion
savepath_vis = '/home/jan/Dokumente/MOBData/Vis/Factorizations' #where to save visualization of decomposition
cfgfile = os.path.join(basepath, 'configfiles', 'decompose', 'nnmf_50_sm2_convex_negTimelowSP.ini') #'sica_200.ini') #
datafile = 'sph_meas'#'ios_meas'
response_window = (8,12) #define frames to calculate odor response; ios:(3,5) sph:(8,12)

In [6]:
animal = '120217FRVg'

### NMF factorization

Performs NMF factorization according to config file. The sparsness parameter is choosen iterativly: First start with 'sparse_start' as initial guess. Sparseness will increase in steps of 'sparse_increase' until spatial component correlation of stimulus dependent components drops below 0.5

In [8]:
sparse_start = 0.2 #inital sparseness strength
assert 'nnmf' in cfgfile

    
#load config
cfg = ConfigObj(cfgfile, unrepr=True)



#load data
filename = os.path.join(datapath, animal, datafile)
ts = ia.TimeSeries()
ts.load(filename)

cfg['sparse_param'] = sparse_start
cfg['verbose'] = 1
cfg['smooth_param'] = 5
cfg['num_components'] = 20

# perform decomposition
decomposer = ia.NNMF(**cfg)
decomposition = decomposer(ts)
decomposition.base._series[np.isnan(decomposition.base._series)] = 0 #clear nans
    
# calc spatial cor of stimulus driven components
signal = ia.TrialMean()(ia.CutOut(response_window)(decomposition))
mode_cor = ia.CalcStimulusDrive()(signal)
mask = mode_cor._series.squeeze()<0.5
if np.sum(mask) != 0: #if there are stimulus driven components  
    selected_modes = ia.SelectObjects()(decomposition, mask)   
    cor = np.nanmax(1-pdist(selected_modes.base._series, 'correlation'))
else:
    cor = np.nanmax(1-pdist(decomposition.base._series, 'correlation'))

FileNotFoundError: [Errno 2] No such file or directory: '/media/jan/BackupWork/Documents/NewAnalysis/MOBconverted/120217FRVg/sph_meas.json'

In [ ]:
cfg

In [15]:
ax = plt.subplot(111)
for ix, base in enumerate(decomposition.base.shaped2D()):
    mycolors = ['c', 'b'] if mode_cor._series.squeeze()[ix] <0.4 else ['0.3', '0.7']
    if decomposition.name.split('_')[1] == 'l':
        base  = base[::-1]    
    ax.contourf(base, [0.3,0.7,1], colors=mycolors, alpha=0.5)
plt.show()

In [16]:
print(np.nanmax(1-pdist(decomposition.base._series, 'correlation')))


for ix, base in enumerate(decomposition.base.shaped2D()):
    ax = plt.subplot(8,7,ix+1)
    ax.imshow(base, vmin=0, vmax=1, interpolation='none')
plt.show()

0.439238220935


### sICA factorization

In [28]:
redo = True
assert 'sica' in cfgfile
animals = [ani for ani in os.listdir(datapath) if ('FRV' in ani)] #('FRV' not in ani)]

for animal in animals:
    
    savelocation = os.path.join(savepath, animal)
    savename = os.path.splitext(os.path.basename(cfgfile))[0] + '_' + datafile
    if os.path.exists(savelocation):
        if os.path.exists(os.path.join(savelocation, savename + '.npy')) and not(redo):
            print('%s already done'%animal)
            continue
    else:
        os.makedirs(savelocation)
        
    filename = os.path.join(datapath, animal, datafile)
    ts = ia.TimeSeries()
    try:
        ts.load(filename)
    except IOError:
        print('!!! No data for animal %s !!!'%animal)
        continue
    cfg = ConfigObj(cfgfile, unrepr=True)
        
    # perform decomposition
    decomposer = ia.sICA(**cfg)
    decomposition = decomposer(ts)
    decomposition.base._series[np.isnan(decomposition.base._series)] = 0 #clear nans
   
    decomposition.save(os.path.join(savelocation, savename))
    print('%s done'%animal)

{'num_components': 200}
110406 done
{'num_components': 200}
090402r done
{'num_components': 200}
110401 done
{'num_components': 200}
110830 done
{'num_components': 200}
090810 done
{'num_components': 200}
110210 done
{'num_components': 200}
110616c done
{'num_components': 200}
110615 done
{'num_components': 200}
090414l done
{'num_components': 200}
110916 done
{'num_components': 200}
110610 done
{'num_components': 200}
100215 done
{'num_components': 200}
100715 done
{'num_components': 200}
110627 done
{'num_components': 200}
110519 done
{'num_components': 200}
110314 done
{'num_components': 200}
110913 done
{'num_components': 200}
110329 done
{'num_components': 200}
110505 done
{'num_components': 200}
110331 done
{'num_components': 200}
100225 done
{'num_components': 200}
110504 done
{'num_components': 200}
120107 done
{'num_components': 200}
090820 done
{'num_components': 200}
101126 done
{'num_components': 200}
090331l done
{'num_components': 200}
111222sph done
{'num_components': 20

MemoryError: 

### Plot decomposition overview

Plot decompositions for selected animals

In [30]:
method = 'sica_200_ios_meas'#'nnmf_200_sm2_convex_negTimelowSP_sp*_ios_meas'
draw_sphrois = True
animals = [ani for ani in os.listdir(savepath) if ('FRV' not in ani)]

In [31]:
rows = np.ceil(len(animals)/5.)
fig = plt.figure(figsize=(10, 1.5*int(rows)))
plt.subplots_adjust(left=0.02, bottom=0.02, right=0.98, top=0.9)

for ix, animal in enumerate(animals):
    ax = fig.add_subplot(rows, 5, ix+1)
    bg = Image.open(os.path.join(datapath, animal, 'bg.png'))
    bg = bg.convert('L')
    
    mf = ia.TimeSeries()
    filename = glob.glob(os.path.join(savepath, animal, method+'.npy'))
    assert len(filename) == 1
    filename = filename[0].split('.')[0]
    mf.load(filename)
    
    bg = bg.resize(mf.base.shape[::-1])
    bg = np.asarray(bg)
    if mf.name.split('_')[1] == 'l':
        bg = bg[::-1]    
       
    mf = ia.TrialMean()(ia.CutOut(response_window)(mf))
    t2t = ia.CalcStimulusDrive()(mf)._series.squeeze()
    
    myextent = np.array([0, mf.base.shape[1], mf.base.shape[0], 0])-0.5
    ax.imshow(bg, interpolation='none', extent=myextent, cmap=plt.cm.bone)
    for ix, base in enumerate(mf.base.shaped2D()):
        mycolors = ['c', 'b'] if t2t[ix] <0.4 else ['0.3', '0.7']
        if mf.name.split('_')[1] == 'l':
            base  = base[::-1]    
        ax.contourf(base, [0.3,0.7,1], colors=mycolors, alpha=0.5)
    
    #show rois
    roi_path = os.path.join(datapath, animal, 'rois')
    if draw_sphrois and os.path.exists(roi_path+'.npy'):
        rois = ia.TimeSeries()
        rois.load(roi_path)
        for roi in rois.shaped2D():
            if mf.name.split('_')[1] == 'l':
                roi = roi[::-1]
            ax.contour(roi, [0.5], colors=['w'], lw=2)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(animal, size=8)
    
fig.savefig(os.path.join(savepath_vis, method+'.pdf'))
plt.show()

AttributeError: 'NoneType' object has no attribute 'tk'

!!! warning: no repeated stimuli!!!
